# ClutchAI MVP

__GOAL__ 

Create an Agentic AI assistant for Yahoo Fantasy League.

__STEPS__
1. Add YahooFantasy API as a Tool
2. Create a VectorDB from Josh Llyod Youtube Transcripts
3. Run a ReACT Agent 
4. Test out key usecases:
    - Give me a weekly summary from last week
    - Who is more likely to win in week A vs week B
    - Who is hot on the waiver wire?
    - Is player A for player B a good trade?

#### 0. Environment Setup

In [31]:
import os
from dotenv import load_dotenv
from pathlib import Path
import json

# Load .env file from the project root (parent directory of notebook/)
# Resolve the path to get the absolute path to the .env file
env_file_location = Path('..').resolve()
load_dotenv(dotenv_path=env_file_location / ".env")
print(f"Loading .env from: {env_file_location / '.env'}\n")

# Set env variables
YAHOO_CONSUMER_KEY = os.environ.get('YAHOO_CLIENT_ID', "<INSERT_>")
YAHOO_CONSUMER_SECRET = os.environ.get('YAHOO_CLIENT_SECRET', "<INSERT>")
YAHOO_LEAGUE_ID = 58930
GAME_CODE = "nba"
GAME_ID=466

Loading .env from: /Users/matt/Code/ClutchAI/.env



In [32]:
# Show which env variables were loaded
env_vars_to_check = ['YAHOO_CLIENT_ID', 'YAHOO_CLIENT_SECRET', 'OPENAI_API_KEY', 'LANGSMITH_API_KEY']
print("Environment variables loaded:")
for var in env_vars_to_check:
    value = os.environ.get(var)
    if value:
        print(f"  ✓ {var}: {value[:10]} + ...")
    else:
        print(f"  ✗ {var}: NOT SET")

Environment variables loaded:
  ✓ YAHOO_CLIENT_ID: dj0yJmk9Ml + ...
  ✓ YAHOO_CLIENT_SECRET: 967c74034e + ...
  ✓ OPENAI_API_KEY: sk-proj-4A + ...
  ✓ LANGSMITH_API_KEY: lsv2_pt_7c + ...


#### 1. Connect to YahooFantasy League Data

In [ ]:
from yfpy.query import YahooFantasySportsQuery

In [39]:
query = YahooFantasySportsQuery(
    league_id=YAHOO_LEAGUE_ID,
    game_code="nba",
    game_id= GAME_ID,
    env_var_fallback = True,
    env_file_location = env_file_location,
    save_token_data_to_env_file = True,
)

In [40]:
#Data to pull from Yahoo Fantasy
league_meta = query.get_league_metadata()
league_meta_dict = json.loads(league_meta.to_json())
print(len(league_meta))
league_meta_dict


31


{'allow_add_to_dl_extra_pos': 1,
 'current_week': 3,
 'draft_status': 'postdraft',
 'edit_key': '2025-11-06',
 'end_date': '2026-04-05',
 'end_week': 23,
 'felo_tier': 'gold',
 'game_code': 'nba',
 'iris_group_chat_id': None,
 'is_cash_league': 0,
 'is_highscore': False,
 'is_plus_league': 0,
 'is_pro_league': 0,
 'league_id': '58930',
 'league_key': '466.l.58930',
 'league_type': 'private',
 'league_update_timestamp': 1762418787,
 'logo_url': 'https://yahoofantasysports-res.cloudinary.com/image/upload/t_s192sq/fantasy-logos/56479288575_ec9899.jpg',
 'matchup_week': 3,
 'name': 'TK Fiji Fantasy 2024/25',
 'num_teams': 12,
 'renew': '454_14696',
 'renewed': None,
 'roster_type': 'date',
 'scoring_type': 'headone',
 'season': 2025,
 'start_date': '2025-10-21',
 'start_week': 1,
 'url': 'https://basketball.fantasysports.yahoo.com/nba/58930',
 'weekly_deadline': 'intraday'}

#### 2. Connecting LangChain Agent to Yahoo API

In [41]:
from langchain_core.tools import tool
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI

In [42]:
#Creat YahooFantasy Agent
class YahooFantasyAgent:
    def __init__(self):
        self.YAHOO_CONSUMER_KEY = os.environ.get('YAHOO_CLIENT_ID', "<INSERT_>")
        self.YAHOO_CONSUMER_SECRET = os.environ.get('YAHOO_CLIENT_SECRET', "<INSERT>")
        self.YAHOO_LEAGUE_ID = 58930
        self.GAME_CODE = "nba"
        self.GAME_ID=466

        self.query = YahooFantasySportsQuery(
            league_id=YAHOO_LEAGUE_ID,
            game_code="nba",
            game_id= GAME_ID,
            yahoo_consumer_key = YAHOO_CONSUMER_KEY,
            yahoo_consumer_secret = YAHOO_CONSUMER_SECRET,
            env_var_fallback = True,
            env_file_location = env_file_location,
            save_token_data_to_env_file = True,
        )

    def get_league_metadata(self) -> str:
        """Fetch Yahoo Fantasy league metadata."""
        try:
            data = self.query.get_league_metadata()
            return f'The Yahoo Fantasy league metadata in json format is: {data}'
        except Exception as e:
            return f"Failed to Yahoo Fantasy league metadata: {e}"
        
@tool("YahooLeagueMetDataTool", description="Get Yahoo League Metadata in json format from YPFS.")
def league_metadata_tool() -> str:
    """Get Yahoo League Metadata."""
    return YahooFantasyAgent().get_league_metadata()

In [55]:
#Create Clutch AI Agent
llm = ChatOpenAI(model_name="gpt-4")
tools = [league_metadata_tool]

ClutchAI_Agent = create_agent(
    model = llm,
    tools = tools,
    system_prompt="You are a helpful assistant for a Yahoo Fantasy Sports league manager.",
)

In [56]:
#Test Agent
inputs = {"messages": [{"role": "user", "content": "What is the Yahoo Fantasy League Name?"}]}
ClutchAI_Agent.invoke(inputs)

{'messages': [HumanMessage(content='What is the Yahoo Fantasy League Name?', additional_kwargs={}, response_metadata={}, id='be0bc34b-d652-4db6-ad7a-30a203670c98'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 68, 'total_tokens': 79, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-4-0613', 'system_fingerprint': None, 'id': 'chatcmpl-CYyQQXznTyQmZiwvN0GCSHTlwvAEB', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--370c7b62-dd66-48ee-b292-c63fb876e9d7-0', tool_calls=[{'name': 'YahooLeagueMetDataTool', 'args': {}, 'id': 'call_QG4XSHuSd0ATgzO6M4OZw55R', 'type': 'tool_call'}], usage_metadata={'input_tokens': 68, 'output_tokens': 11, 'total_tokens': 79, 'input_

In [11]:
#Stream Agent Response for Debug
for event in ClutchAI_Agent.stream(inputs):
    print(event)

{'model': {'messages': [AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 151, 'prompt_tokens': 151, 'total_tokens': 302, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CYpF2SldRbg7sMyKpV3AQWihSE8Uo', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--6dce8e7f-3aad-46c1-9c44-95d63b3f2efb-0', tool_calls=[{'name': 'YahooLeagueMetDataTool', 'args': {}, 'id': 'call_HfBWvVMEQmUbQwmIohaNWAGz', 'type': 'tool_call'}], usage_metadata={'input_tokens': 151, 'output_tokens': 151, 'total_tokens': 302, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 128}})]}}
{'tools': {'messa

#### 2. Adding Youtube Transcripts to VectorDB

In [47]:
from langchain_community.document_loaders import YoutubeLoader
from langchain_community.document_loaders.youtube import TranscriptFormat
from youtube_transcript_api import YouTubeTranscriptApi

In [50]:
loader = YoutubeLoader.from_youtube_url(
    "https://www.youtube.com/watch?v=TB2QwCRMams&t=2s", 
    add_video_info=False,
    transcript_format=TranscriptFormat.CHUNKS,
    chunk_size_seconds=30
)
loader.load()

[Document(metadata={'source': 'https://www.youtube.com/watch?v=TB2QwCRMams&t=0s', 'start_seconds': 0, 'start_timestamp': '00:00:00'}, page_content="There are a lot of players who are either hurt, injury-prone, susceptible to tanking. So, what do you do in a fantasy draft? When do you take them? Michael Bolton, he's going to give you some answers. >> Thanks, Josh. It's Michael Bolton here, and it's time for another episode of the Locked On Fantasy Basketball Podcast. Let's get to it. >> Let's get to it. Indeed. You are Locked on Fantasy, your daily"),
 Document(metadata={'source': 'https://www.youtube.com/watch?v=TB2QwCRMams&t=30s', 'start_seconds': 30, 'start_timestamp': '00:00:30'}, page_content="NBA fantasy podcast, part of the Locked On podcast network, your team every day. Hello and welcome to the Locked On Fantasy Basketball podcast brought to you by Basketball Monster. My name is Josh Lloyd and there is no longer a hole in my bucket. I'm also the lead fantasy analyst at basketbal